In [3]:
import pylibxc
import numpy as np
import pandas as pd
from pathlib import Path

# path/to/data.csv
data_path = Path('data/comprehensive_search.csv')
# folder for outputs
out_path = Path('out')
out_path.mkdir(exist_ok=True, parents=True)
df = pd.read_csv(data_path)

Generate bibtex file and references for functionals

In [4]:
import re

# Simplified using re..
latex_parse = ['@article{', '@article {', '@inproceedings{', '@Inbook{', '@misc{']


bib_file = []
ref_names = []
is_combined_xc = []
for xc in df['xc_id']:
  try:
    func_c = pylibxc.LibXCFunctional(xc, "polarized")
    combined_xc = False
  except:
    # is an xc functional
    func_c = pylibxc.LibXCFunctional(xc.replace('_c_', '_xc_'), "unpolarized")
    combined_xc = True

  is_combined_xc.append(combined_xc)

  ref_name='\cite{'
  for ref in func_c.get_bibtex():
    if ref not in bib_file:
      bib_file.append(ref)
    latex_cite = ref.splitlines()[0]
    # get latex cite name by removing the following prefixes
    # like @article{, @inproceedings{, @Inbook{, @misc{, etc.
    latex_cite = re.search(r'@.*{(.*)', latex_cite).group(1)  
    ref_name += latex_cite
    
  ref_name = ref_name[:-1] + '}' 

  ref_names.append(ref_name)

df['Refs.'] = ref_names 
df['combined xc'] = is_combined_xc

# write out bib file
bib_file = "\n".join(bib_file)
out_file = out_path / data_path.name.replace('.csv','.bib')
f = open(out_file, "w")
f.write(bib_file)
f.close()


Create latex tables

In [5]:
min_df = df[["xc_id", "xc_name", "Refs.", "condition", "violation fraction"]]
table_df = min_df.pivot_table(values='violation fraction', index=min_df['xc_id'], columns='condition', aggfunc='first')

table_df["table_name"] = min_df.groupby('xc_id').apply(lambda x: (x['xc_name'] + "~" + x['Refs.']).iloc[0])

pretty_conds = {
    'negativity_check': '$E_c$ non-positivity',
    'deriv_lower_bd_check': '$E_c[n_{\gamma}]$ inequalities',
    'deriv_upper_bd_check_1': '$T_c$ upper-bound',
    'second_deriv_check': '$E^{\lambda}_c[n]$ concavity',
    'lieb_oxford_bd_check_Exc': r'LO bound on $E_{xc}$',
    'lieb_oxford_bd_check_Uxc': r'LO bound on $U_{xc}$',
    "deriv_upper_bd_check_2": r'Conjecture: $T_c \leq -E_c$',
}

sorted_cols = ['table_name'] + list(pretty_conds)
table_df = table_df.reindex(sorted_cols, axis=1)
table_df.rename(columns=pretty_conds, inplace=True)
table_df.dropna(inplace=True)

In [6]:

# avoid pandas truncation in latex table
with pd.option_context("max_colwidth", 1000):
    latex = table_df.to_latex(index=False, escape=False, 
        column_format=(len(table_df.columns)-1)*'|c'+ '|l|',
        float_format="%.3f",
        header=[''] + list(table_df.columns[1:]))

# horizontal line sep
latex = latex.replace("\\\n", "\\ \hline\n").replace('\\toprule','\\hline')

latex_list = latex.splitlines()
latex=[]
for line in latex_list:
  if '\\toprule' in line or '\\midrule' in line or '\\bottomrule' in line:
    continue
  else:
    latex.append(line)

latex = "\n".join(latex)

out_file = out_path / data_path.name.replace('.csv','.tex')
f = open(out_file, "w")
f.write(latex)
f.close()

/tmp/ipykernel_6307/380606811.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = table_df.to_latex(index=False, escape=False,
